#  Summary of the code

•	Here I present the different datasets to be used in the data analysis and model building.

•	The dataset containing dissolved nitrogen concentrations was consistent from 2000 to 2017. Therefore, the analysis is constraint to that period of time. 

•	Descriptions about the type of data and how to access it are included.

•	When needed, data is cleaned. For example, when flags within the documents indicate compromised data. 

•	Extra variables are calculated from the different datasets. For example, accumulated days with precipitation from precipitation data.


# Import relevant packages

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import datasets

NFK international airport data (wind/precipitation)

    Data is publicly available at https://www.ncdc.noaa.gov/cdo-web/cart

    Station name: NORFOLK INTERNATIONAL AIRPORT, VA US

    Station ID: GHCND:USW00013737

    Data contains daily average of wind speed and wind direction, and daily values of accumulated precipitation.

In [2]:
infile_NFK_airport='C:/Users/a_mac/Box Sync/current projects/MTM through the year/0_NFK_weather_review/NCDC_NOAA_precipitation_wind_2000_2018_NFK_IntAirp.csv'
NFK_airport=pd.read_csv(infile_NFK_airport)
NFK_airport.head()

,STATION,NAME,DATE,month,day,year,AWND,AWND_ATTRIBUTES,FMTM,FMTM_ATTRIBUTES,...,WDF5,WDF5_ATTRIBUTES,WDFG,WDFG_ATTRIBUTES,WSF2,WSF2_ATTRIBUTES,WSF5,WSF5_ATTRIBUTES,WSFG,WSFG_ATTRIBUTES
0,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",1/1/2000,1,1,2000,7.83,",,X",1344.0,",,X",...,220.0,",,X",NaN,NaN,16.1,",,X",19.9,",,X",NaN,NaN
1,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",1/2/2000,1,2,2000,9.40,",,X",1522.0,",,X",...,220.0,",,X",NaN,NaN,21.0,",,X",23.9,",,X",NaN,NaN
2,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",1/3/2000,1,3,2000,12.97,",,X",1407.0,",,X",...,240.0,",,X",NaN,NaN,21.9,",,X",25.9,",,X",NaN,NaN
3,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",1/4/2000,1,4,2000,19.46,",,X",1441.0,",,X",...,220.0,",,X",NaN,NaN,33.1,",,X",40.9,",,X",NaN,NaN
4,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",1/5/2000,1,5,2000,10.96,",,X",633.0,",,X",...,20.0,",,X",NaN,NaN,25.1,",,X",34.0,",,X",NaN,NaN


    Calculate ACCUMULATED precipitation per 'n' days

In [3]:
NFK_airport['DATE'] = pd.to_datetime(NFK_airport['DATE'])
df = NFK_airport.groupby('STATION').apply(lambda x: x.set_index('DATE').resample('1D').first())
# print (df)

# define here the number of days of accumulated rain 
# use 'window = n'
df1 = df.groupby(level=0)['PRCP'].apply(lambda x: x.shift().rolling(min_periods=1,window=2).sum()).reset_index(name='Sum_Past_2_days')
df2 = df.groupby(level=0)['PRCP'].apply(lambda x: x.shift().rolling(min_periods=1,window=3).sum()).reset_index(name='Sum_Past_3_days')
df3 = df.groupby(level=0)['PRCP'].apply(lambda x: x.shift().rolling(min_periods=1,window=5).sum()).reset_index(name='Sum_Past_5_days')

# merged the data
merged_1=pd.merge(NFK_airport,df1)
merged_2=pd.merge(merged_1,df2)
merged_NFK_airport=pd.merge(merged_2,df3)

# take a look on the new table (with the merged data)
merged_NFK_airport.head()

,STATION,NAME,DATE,month,day,year,AWND,AWND_ATTRIBUTES,FMTM,FMTM_ATTRIBUTES,...,WDFG_ATTRIBUTES,WSF2,WSF2_ATTRIBUTES,WSF5,WSF5_ATTRIBUTES,WSFG,WSFG_ATTRIBUTES,Sum_Past_2_days,Sum_Past_3_days,Sum_Past_5_days
0,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",2000-01-01,1,1,2000,7.83,",,X",1344.0,",,X",...,NaN,16.1,",,X",19.9,",,X",NaN,NaN,NaN,NaN,NaN
1,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",2000-01-02,1,2,2000,9.40,",,X",1522.0,",,X",...,NaN,21.0,",,X",23.9,",,X",NaN,NaN,0.00,0.00,0.00
2,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",2000-01-03,1,3,2000,12.97,",,X",1407.0,",,X",...,NaN,21.9,",,X",25.9,",,X",NaN,NaN,0.00,0.00,0.00
3,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",2000-01-04,1,4,2000,19.46,",,X",1441.0,",,X",...,NaN,33.1,",,X",40.9,",,X",NaN,NaN,0.00,0.00,0.00
4,USW00013737,"NORFOLK INTERNATIONAL AIRPORT, VA US",2000-01-05,1,5,2000,10.96,",,X",633.0,",,X",...,NaN,25.1,",,X",34.0,",,X",NaN,NaN,0.65,0.65,0.65


Conductivity and Water temperature from the NOAA station at Sewells Point

    Data is publicly available at https://tidesandcurrents.noaa.gov/physocean.html?bdate=20190101&edate=20191231&units=metric&timezone=GMT&id=8638610&interval=h&action=data

    Station name: Sewells Point, VA

    Station ID: 8638610

    Data contains hourly measurements of water temperature and conductivity

    The page only allows you to download 365 days of data. Therefore, you have to download manually each year if more than one is require.

In [4]:
infile_temp_cond_Sewells='C:/Users/a_mac/Box Sync/current projects/MTM through the year/0_NFK_weather_review/WatTemp_Conduct_Sewellspoint_2000to2017_filled.csv'
data_temp_cond_Sewells=pd.read_csv(infile_temp_cond_Sewells)
# data_temp_cond_Sewells.head()
# data_temp_cond_Sewells.columns
data_temp_cond_Sewells

,Date,month,day,year,Time_GMT,WatTemp_C,Conduct_mS_cm
0,1/1/2000,1,1,2000,0.000000,6.3,NaN
1,1/1/2000,1,1,2000,0.041667,6.3,NaN
2,1/1/2000,1,1,2000,0.083333,6.3,NaN
3,1/1/2000,1,1,2000,0.125000,6.3,NaN
4,1/1/2000,1,1,2000,0.166667,6.2,NaN
5,1/1/2000,1,1,2000,0.208333,6.2,NaN
6,1/1/2000,1,1,2000,0.250000,6.0,NaN
7,1/1/2000,1,1,2000,0.291667,6.1,NaN
8,1/1/2000,1,1,2000,0.333333,6.1,NaN
9,1/1/2000,1,1,2000,0.375000,6.0,NaN


Extract/calculate daily values

In [7]:
OutputFileName="fSewells20002019_temp_cond_perday.csv"
OutputFile=open(OutputFileName,"w")
OutputFile.write("Date,day,month,year,Temp,Cond\n")

for i in data_temp_cond_Sewells.Date.unique():
    Date=i
    day=np.mean(data_temp_cond_Sewells.day[data_temp_cond_Sewells.Date==i])
    month=np.mean(data_temp_cond_Sewells.month[data_temp_cond_Sewells.Date==i])
    year=np.mean(data_temp_cond_Sewells.year[data_temp_cond_Sewells.Date==i])
    Temp=np.max(data_temp_cond_Sewells.WatTemp_C[data_temp_cond_Sewells.Date==i])
    Cond=np.max(data_temp_cond_Sewells.Conduct_mS_cm[data_temp_cond_Sewells.Date==i])
    
    Ouputstring="%s,%i,%i,%i,%f,%f" % (Date,day,month,year,Temp,Cond)
    OutputFile.write(Ouputstring + "\n")
OutputFile.close()

In [8]:
Sewells_temp_cond_perday=pd.read_csv(OutputFileName)
Sewells_temp_cond_perday

,Date,day,month,year,Temp,Cond
0,1/1/2000,1,1,2000,6.9,NaN
1,1/2/2000,2,1,2000,7.5,NaN
2,1/3/2000,3,1,2000,8.0,NaN
3,1/4/2000,4,1,2000,8.9,NaN
4,1/5/2000,5,1,2000,9.0,NaN
5,1/6/2000,6,1,2000,8.7,NaN
6,1/7/2000,7,1,2000,8.8,NaN
7,1/8/2000,8,1,2000,8.6,NaN
8,1/9/2000,9,1,2000,8.6,NaN
9,1/10/2000,10,1,2000,8.8,NaN


In [9]:
Sewells_temp_cond_perday['Date'] = pd.to_datetime(Sewells_temp_cond_perday['Date'])
df = Sewells_temp_cond_perday.groupby('year').apply(lambda x: x.set_index('Date').resample('1D').first())
# print (df)

# define here the number of days of accumulated rain 
# use 'window = n'
df1 = df.groupby(level=0)['Temp'].apply(lambda x: x.shift().rolling(min_periods=1,window=2).sum()).reset_index(name='Sum_Past_2_days')
df2 = df.groupby(level=0)['Temp'].apply(lambda x: x.shift().rolling(min_periods=1,window=3).sum()).reset_index(name='Sum_Past_3_days')
df3 = df.groupby(level=0)['Temp'].apply(lambda x: x.shift().rolling(min_periods=1,window=5).sum()).reset_index(name='Sum_Past_5_days')

# merged the data
merged_1=pd.merge(Sewells_temp_cond_perday,df1)
merged_2=pd.merge(merged_1,df2)
merged_Sewells=pd.merge(merged_2,df3)

# take a look on the new table (with the merged data)
merged_Sewells.head()

,Date,day,month,year,Temp,Cond,Sum_Past_2_days,Sum_Past_3_days,Sum_Past_5_days
0,2000-01-01,1,1,2000,6.9,NaN,NaN,NaN,NaN
1,2000-01-02,2,1,2000,7.5,NaN,6.9,6.9,6.9
2,2000-01-03,3,1,2000,8.0,NaN,14.4,14.4,14.4
3,2000-01-04,4,1,2000,8.9,NaN,15.5,22.4,22.4
4,2000-01-05,5,1,2000,9.0,NaN,16.9,24.4,31.3


Water level from the NOAA station at Sewells Point

    Data is publicly available at https://tidesandcurrents.noaa.gov/physocean.html?bdate=20190101&edate=20191231&units=metric&timezone=GMT&id=8638610&interval=h&action=data

    Station name: Sewells Point, VA

    Station ID: 8638610

    Data contains hourly measurements of water level

    The page only allows you to download 365 days of data. Therefore, you have to download manually each year if more than one is require.

In [10]:
infile_MLLW_Sewells='C:/Users/a_mac/Box Sync/current projects/MTM through the year/0_NFK_weather_review/MLLW_sewellspoint_2000_20018.csv'
MLLW_Sewells=pd.read_csv(infile_MLLW_Sewells)
MLLW_Sewells.head()
# MLLW_Sewells

,Counter,Date,month,day,year,Time_GMT,Predicted_ft,preliminary_ft,Verified_ft
0,1,1/1/2000,1,1,2000,0.000000,1.346,-,0.93
1,2,1/1/2000,1,1,2000,0.041667,0.853,-,0.47
2,3,1/1/2000,1,1,2000,0.083333,0.387,-,0.01
3,4,1/1/2000,1,1,2000,0.125000,0.077,-,-0.27
4,5,1/1/2000,1,1,2000,0.166667,0.017,-,-0.41


# hours above 3ft - MLLW

    filter data for desired water level (MLLW) limit

In [11]:
MLLW_abv_3=MLLW_Sewells[MLLW_Sewells.Verified_ft>2.99]
MLLW_abv_3.head()

,Counter,Date,month,day,year,Time_GMT,Predicted_ft,preliminary_ft,Verified_ft
443,444,1/19/2000,1,19,2000,0.458333,2.673,-,3.14
444,445,1/19/2000,1,19,2000,0.500000,2.951,-,3.36
445,446,1/19/2000,1,19,2000,0.541667,2.849,-,3.23
468,469,1/20/2000,1,20,2000,0.500000,2.777,-,3.17
469,470,1/20/2000,1,20,2000,0.541667,3.048,-,3.54


    Calcute hours above 3ft - MLLW

In [12]:
MLLW_abv_3.columns

Index(['Counter', 'Date', 'month', 'day', 'year', 'Time_GMT', 'Predicted_ft',
       'preliminary_ft', 'Verified_ft'],
      dtype='object')

In [15]:
OutputFileName_MLLWmax="TimeAbove3ft20002019.csv"
OutputFile_MLLWmax=open(OutputFileName_MLLWmax,"w")
OutputFile_MLLWmax.write("Date,month,day,year,above\n")

for i in MLLW_abv_3.Date.unique():
    Date=i
    month=np.mean(MLLW_abv_3.month[MLLW_abv_3.Date==i])
    day=np.mean(MLLW_abv_3.day[MLLW_abv_3.Date==i])
    year=np.mean(MLLW_abv_3.year[MLLW_abv_3.Date==i])
    above=np.sum(MLLW_abv_3.Verified_ft[MLLW_abv_3.Date==i])
    
    Ouputstring_MLLWmax= "%s,%i,%i,%i,%f" % (Date,month,day,year,above)
    OutputFile_MLLWmax.write(Ouputstring_MLLWmax + "\n")

In [16]:
MLLW_above3ft=pd.read_csv(OutputFileName_MLLWmax)
MLLW_above3ft

,Date,month,day,year,above
0,1/19/2000,1,19,2000,9.73
1,1/20/2000,1,20,2000,13.41
2,1/21/2000,1,21,2000,15.95
3,1/22/2000,1,22,2000,3.02
4,1/24/2000,1,24,2000,9.62
5,1/25/2000,1,25,2000,78.91
6,1/26/2000,1,26,2000,6.11
7,3/12/2000,3,12,2000,6.06
8,3/18/2000,3,18,2000,6.16
9,3/19/2000,3,19,2000,6.48


# Accumulated MLLW per days

    Extract/calcuate the highest water level per day

In [19]:
OutputFileName_MLLWmax="max20002017MLLWperday.csv"
OutputFile_MLLWmax=open(OutputFileName_MLLWmax,"w")
OutputFile_MLLWmax.write("date,month,day,year,mean_ft,max_ft\n")

for i in MLLW_Sewells.Date.unique():
    maximMLLW=np.max(MLLW_Sewells.Verified_ft[MLLW_Sewells.Date==i])
    Date=i
    month=np.mean(MLLW_Sewells.month[MLLW_Sewells.Date==i])
    day=np.mean(MLLW_Sewells.day[MLLW_Sewells.Date==i])
    year=np.mean(MLLW_Sewells.year[MLLW_Sewells.Date==i])
    avgMLLW=np.mean(MLLW_Sewells.Verified_ft[MLLW_Sewells.Date==i])
    Ouputstring_MLLWmax= "%s,%i,%i,%i,%f,%f" % (Date,month,day,year,avgMLLW,maximMLLW)
    OutputFile_MLLWmax.write(Ouputstring_MLLWmax + "\n")

In [20]:
MLLW_Sewells_max=pd.read_csv(OutputFileName_MLLWmax)
MLLW_Sewells_max

,date,month,day,year,mean_ft,max_ft
0,1/1/2000,1,1,2000,0.965417,2.31
1,1/2/2000,1,2,2000,1.019583,2.41
2,1/3/2000,1,3,2000,0.669167,2.07
3,1/4/2000,1,4,2000,0.462500,1.80
4,1/5/2000,1,5,2000,0.891250,2.75
5,1/6/2000,1,6,2000,0.637917,2.04
6,1/7/2000,1,7,2000,0.900417,2.34
7,1/8/2000,1,8,2000,0.921250,2.31
8,1/9/2000,1,9,2000,0.712083,2.14
9,1/10/2000,1,10,2000,1.038333,2.53


    Calculate ACCUMULATED water level per 'n' days

In [21]:
MLLW_Sewells_max['date'] = pd.to_datetime(MLLW_Sewells_max['date'])
df = MLLW_Sewells_max.groupby('year').apply(lambda x: x.set_index('date').resample('1D').first())
# print (df)

# define here the number of days of accumulated rain 
# use 'window = n'
df1 = df.groupby(level=0)['max_ft'].apply(lambda x: x.shift().rolling(min_periods=1,window=2).sum()).reset_index(name='Sum_Past_2_days')
df2 = df.groupby(level=0)['max_ft'].apply(lambda x: x.shift().rolling(min_periods=1,window=3).sum()).reset_index(name='Sum_Past_3_days')
df3 = df.groupby(level=0)['max_ft'].apply(lambda x: x.shift().rolling(min_periods=1,window=5).sum()).reset_index(name='Sum_Past_5_days')

# merged the data
merged_1=pd.merge(MLLW_Sewells_max,df1)
merged_2=pd.merge(merged_1,df2)
merged_MLLW_Sewells=pd.merge(merged_2,df3)

# take a look on the new table (with the merged data)
merged_MLLW_Sewells

,date,month,day,year,mean_ft,max_ft,Sum_Past_2_days,Sum_Past_3_days,Sum_Past_5_days
0,2000-01-01,1,1,2000,0.965417,2.31,NaN,NaN,NaN
1,2000-01-02,1,2,2000,1.019583,2.41,2.31,2.31,2.31
2,2000-01-03,1,3,2000,0.669167,2.07,4.72,4.72,4.72
3,2000-01-04,1,4,2000,0.462500,1.80,4.48,6.79,6.79
4,2000-01-05,1,5,2000,0.891250,2.75,3.87,6.28,8.59
5,2000-01-06,1,6,2000,0.637917,2.04,4.55,6.62,11.34
6,2000-01-07,1,7,2000,0.900417,2.34,4.79,6.59,11.07
7,2000-01-08,1,8,2000,0.921250,2.31,4.38,7.13,11.00
8,2000-01-09,1,9,2000,0.712083,2.14,4.65,6.69,11.24
9,2000-01-10,1,10,2000,1.038333,2.53,4.45,6.79,11.58


Biochemicall data (i.e. dissolved nitrogen) from the Virginia Estuarine and Coastal Observing System station at the mouth of the Lafayete River.

    Data is publicly available at http://vecos.vims.edu/

    Station name: Lafayette - Chesapeake Bay Program Longterm Monitoring Station 

    Station ID: LAF1

    one sample collection multiple times per month (inconsistently). For a given day, data is available at different water-column depths.

In [22]:
infile_VECOS_Laf_head='C:/Users/a_mac/Box Sync/current projects/MTM through the year/0_NFK_weather_review/VECOS Lafayette/VECOS head lafayette_Granby_ long term monitoring station 1998 to 2018.csv'
VECOS_Laf_head=pd.read_csv(infile_VECOS_Laf_head)
VECOS_Laf_head

,Station,EventID,Cruise,Program,Project,Agency,Source,Station.1,date,month,...,value,unit,Method,Lab,Problem,Precision,BiasPC,Details,Lat,Long
0,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,0.13000,MG/L,D01,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144
1,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,0.16000,MG/L,D01,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144
2,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,10.14000,MG/L,F01,NaN,NaN,NaN,NaN,STORET DATA CONVERSION,36.8894,-76.28144
3,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,10.16000,MG/L,F01,NaN,NaN,NaN,NaN,STORET DATA CONVERSION. DEPTH DESCRIPTION AND ...,36.8894,-76.28144
4,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,10.19000,MG/L,F01,NaN,NaN,NaN,NaN,STORET DATA CONVERSION,36.8894,-76.28144
5,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,0.27700,MG/L,D02,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144
6,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,0.36500,MG/L,D02,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144
7,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,0.00100,MG/L,D01,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144
8,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,0.01000,MG/L,D01,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144
9,LFB01,122199,BAY272,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,1/22/1998,1,...,240.00000,MPN/100 ML,L01,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144


In [23]:
VECOS_filter=VECOS_Laf_head[(VECOS_Laf_head.year>1999)&(VECOS_Laf_head.depth<1.5)&(VECOS_Laf_head.Parameter == "NO23F")]

Calculate average dissolved nitrogen concetration to be used in the anomaly

In [24]:
VECOS_filter['date'] = pd.to_datetime(VECOS_filter['date'])
df = VECOS_filter.groupby('year').apply(lambda x: x.set_index('date').resample('1D').first())
# print (df)

# define here the number of days of accumulated rain 
# use 'window = n'
df1 = df.groupby(level=0)['value'].apply(lambda x: x.shift().rolling(min_periods=1,window=30).mean()).reset_index(name='mean_Past_30_days')
df2 = df.groupby(level=0)['value'].apply(lambda x: x.shift().rolling(min_periods=1,window=40).mean()).reset_index(name='mean_Past_40_days')
df3 = df.groupby(level=0)['value'].apply(lambda x: x.shift().rolling(min_periods=1,window=50).mean()).reset_index(name='mean_Past_50_days')

# merged the data
merged_1=pd.merge(VECOS_filter,df1)
merged_2=pd.merge(merged_1,df2)
merged_VECOS=pd.merge(merged_2,df3)

# take a look on the new table (with the merged data)
merged_VECOS

C:\Users\a_mac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Station,EventID,Cruise,Program,Project,Agency,Source,Station.1,date,month,...,Lab,Problem,Precision,BiasPC,Details,Lat,Long,mean_Past_30_days,mean_Past_40_days,mean_Past_50_days
0,LFB01,110013,BAY312,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-01-31,1,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,NaN,NaN,NaN
1,LFB01,110259,BAY313,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-02-24,2,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,0.035,0.035,0.035
2,LFB01,110552,BAY314,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-03-23,3,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,0.006,0.006,0.006
3,LFB01,111328,BAY319,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-05-25,5,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,NaN,NaN,NaN
4,LFB01,96942,BAY320,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-06-22,6,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,0.004,0.004,0.004
5,LFB01,97265,BAY322,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-07-20,7,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,0.004,0.004,0.004
6,LFB01,97725,BAY324,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-08-23,8,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,NaN,0.004,0.004
7,LFB01,98036,BAY326,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-09-21,9,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,0.008,0.008,0.008
8,LFB01,98404,BAY328,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-10-26,10,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,NaN,0.048,0.048
9,LFB01,98651,BAY330,TWQM,TRIB,VADEQ,VADEQ/TRO,LFB01,2000-11-27,11,...,VADCLS,NaN,NaN,NaN,NaN,36.8894,-76.28144,NaN,0.005,0.005


In [25]:
Calculate dissolved nitrogen anomaly

SyntaxError: invalid syntax (<ipython-input-25-0bcd16be4e86>, line 1)

In [28]:
OutputFileName = "anomalyNOx_VECOS.csv"
OutputFile = open(OutputFileName,"w")
OutputFile.write("EventID,month,day,year,anomNOx,nitra,avgnitr\n")

for i in merged_VECOS.date[(merged_VECOS.Parameter == "NO23F")].unique():
    EventID=np.mean(merged_VECOS.EventID[merged_VECOS.date==i])
#     Date=pd.to_datetime(str(merged_VECOS.date[merged_VECOS.date==i]))
#     Date=VECOS_filter.date[VECOS_filter.date==i]
#     Date=merged_VECOS.date[merged_VECOS.date==i].unique()
#     Date=merged_VECOS.date[merged_VECOS.date==i]
#     Date==i.unique()
    month=np.mean(merged_VECOS.month[merged_VECOS.date==i])
    day=np.mean(merged_VECOS.day[merged_VECOS.date==i])
    year=np.mean(merged_VECOS.year[merged_VECOS.date==i])
    anomNOx=np.mean((merged_VECOS.value[merged_VECOS.date==i])-(merged_VECOS.mean_Past_50_days[merged_VECOS.date==i]))
    nitr=np.mean(merged_VECOS.value[merged_VECOS.date==i])
    avgnitr=np.mean(merged_VECOS.mean_Past_50_days[merged_VECOS.date==i])
    
    Outputstring = "%i,%i,%i,%i,%f,%f,%f" % (EventID,month,day,year,anomNOx,nitr,avgnitr)
    OutputFile.write(Outputstring + "\n")

In [29]:
infile = "anomalyNOx_VECOS.csv"
anomalyNOx_VECOS=pd.read_csv(infile)
anomalyNOx_VECOS

,EventID,month,day,year,anomNOx,nitra,avgnitr
0,110013,1,31,2000,NaN,0.0350,NaN
1,110259,2,24,2000,-0.029,0.0060,0.035
2,110552,3,23,2000,0.036,0.0420,0.006
3,111328,5,25,2000,NaN,0.0040,NaN
4,96942,6,22,2000,0.000,0.0040,0.004
5,97265,7,20,2000,0.000,0.0040,0.004
6,97725,8,23,2000,0.004,0.0080,0.004
7,98036,9,21,2000,0.040,0.0480,0.008
8,98404,10,26,2000,-0.043,0.0050,0.048
9,98651,11,27,2000,0.012,0.0170,0.005


# Combine the datasets (nutrients, wind/precipitation, and water level) for matching dates



Build a code to match the days in which all data is available



In [32]:
OutputFileName = "anomalyNOxmatchingdays.csv"
OutputFile = open(OutputFileName,"w")
OutputFile.write("month,day,year,nitra,anomNOx,rain,tworain,threerain,fiverain,windspeed,winddir,Condct,WatTemp,twoWatTemp,threeWatTemp,fiveWatTemp,aboveMLLW,MLLW,twoMLLW,threeMLLW,fiveMLLW\n")

for i in VECOS_filter.date[(VECOS_filter.Parameter == "NO23F")].unique():
    date=NFK_airport.DATE[NFK_airport.DATE==i]
    month=NFK_airport.month[NFK_airport.DATE==i]
    day=NFK_airport.day[NFK_airport.DATE==i]
    year=NFK_airport.year[NFK_airport.DATE==i]
    EventID=np.mean(VECOS_filter.EventID[VECOS_filter.date==i])
    nitra=np.mean(anomalyNOx_VECOS.nitra[anomalyNOx_VECOS.EventID==EventID])
    anomNOx=np.mean(anomalyNOx_VECOS.anomNOx[anomalyNOx_VECOS.EventID==EventID])
    

    rain=merged_NFK_airport.PRCP[merged_NFK_airport.DATE==i]
    tworain=merged_NFK_airport.Sum_Past_2_days[merged_NFK_airport.DATE==i]
    threerain=merged_NFK_airport.Sum_Past_3_days[merged_NFK_airport.DATE==i]
    fiverain=merged_NFK_airport.Sum_Past_5_days[merged_NFK_airport.DATE==i]
    windspeed=merged_NFK_airport.AWND[merged_NFK_airport.DATE==i]
    winddir=merged_NFK_airport.WDF2[merged_NFK_airport.DATE==i]
    Condct=np.mean(merged_Sewells.Cond[merged_Sewells.Date==i])
    WatTemp=np.mean(merged_Sewells.Temp[merged_Sewells.Date==i])
    twoWatTemp=np.mean(merged_Sewells.Sum_Past_2_days[merged_Sewells.Date==i])
    threeWatTemp=np.mean(merged_Sewells.Sum_Past_3_days[merged_Sewells.Date==i])
    fiveWatTemp=np.mean(merged_Sewells.Sum_Past_5_days[merged_Sewells.Date==i])
    aboveMLLW=np.max(MLLW_above3ft.above[MLLW_above3ft.Date==i])
    MLLW=np.mean(merged_MLLW_Sewells.max_ft[merged_MLLW_Sewells.date==i])
    twoMLLW=np.mean(merged_MLLW_Sewells.Sum_Past_2_days[merged_MLLW_Sewells.date==i])
    threeMLLW=np.mean(merged_MLLW_Sewells.Sum_Past_3_days[merged_MLLW_Sewells.date==i])
    fiveMLLW=np.mean(merged_MLLW_Sewells.Sum_Past_5_days[merged_MLLW_Sewells.date==i])



    Outputstring = "%i,%i,%i,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f" % (month,day,year,nitra,anomNOx,rain,tworain,threerain,fiverain,windspeed,winddir,Condct,WatTemp,twoWatTemp,threeWatTemp,fiveWatTemp,aboveMLLW,MLLW,twoMLLW,threeMLLW,fiveMLLW)

    OutputFile.write(Outputstring + "\n")

In [33]:
infile = "anomalyNOxmatchingdays.csv"
match_data=pd.read_csv(infile)
match_data

,month,day,year,nitra,anomNOx,rain,tworain,threerain,fiverain,windspeed,...,Condct,WatTemp,twoWatTemp,threeWatTemp,fiveWatTemp,aboveMLLW,MLLW,twoMLLW,threeMLLW,fiveMLLW
0,1,31,2000,0.0350,NaN,0.00,1.07,1.07,1.07,8.95,...,NaN,1.800000,2.900000,4.400000,9.300000,NaN,2.16,3.26,4.91,10.26
1,2,24,2000,0.0060,-0.029,0.00,0.00,0.00,0.02,9.40,...,NaN,7.300000,13.100000,19.400000,31.600000,NaN,2.11,4.49,7.20,12.83
2,3,23,2000,0.0420,0.036,0.00,0.67,0.81,0.81,10.07,...,NaN,12.150000,21.400000,32.500000,56.100000,NaN,4.25,8.35,12.07,18.45
3,5,25,2000,0.0040,NaN,0.10,0.00,0.68,1.11,7.83,...,NaN,22.300000,42.200000,64.275000,106.675000,NaN,2.79,6.02,9.24,15.48
4,6,22,2000,0.0040,0.000,0.02,0.01,0.81,1.12,12.30,...,NaN,25.500000,50.300000,76.100000,127.500000,NaN,2.12,5.25,7.61,12.64
5,7,20,2000,0.0040,0.000,0.21,0.99,0.99,2.43,7.38,...,NaN,26.600000,53.900000,80.000000,133.175000,NaN,2.98,5.85,9.04,15.30
6,8,23,2000,0.0080,0.004,0.00,0.00,0.00,0.75,7.38,...,NaN,24.900000,50.300000,75.500000,126.700000,NaN,2.56,5.60,8.65,14.60
7,9,21,2000,0.0480,0.040,0.00,0.00,0.23,0.23,8.95,...,NaN,23.200000,46.100000,69.500000,118.300000,NaN,2.44,4.77,7.30,14.08
8,10,26,2000,0.0050,-0.043,0.00,0.00,0.00,0.00,8.50,...,NaN,NaN,NaN,NaN,NaN,NaN,3.04,5.93,8.98,14.52
9,11,27,2000,0.0170,0.012,0.00,0.73,0.73,0.73,7.83,...,NaN,NaN,NaN,NaN,NaN,NaN,2.71,6.50,9.01,12.74
